In [ ]:
"""
Experimento: Decomposição do Erro com Restrições Soft nos Coeficientes Polinomiais

Este experimento simula a decomposição do erro (viés², variância e erro irredutível)
de modelos polinomiais ajustados a dados sintéticos com ruído. A complexidade do modelo
é controlada por um hiperparâmetro `alpha`, que define a penalização sobre os coeficientes
do polinômio. Quanto menor o `alpha`, mais restritivos são os pesos de penalização, o que
favorece modelos mais simples. O ajuste é feito por minimização de uma função de perda
penalizada, sem restrições explícitas (penalização soft).
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Define a função verdadeira que gera os dados
def f_true(x):
    return 2 * x + np.cos(4 * np.pi * x)

# Gera a base polinomial até um determinado grau
def poly_basis(x, degree):
    return np.vstack([x**k for k in range(degree + 1)]).T

# -------------------------------
# PARÂMETROS DO EXPERIMENTO
# -------------------------------

np.random.seed(42)           # Reprodutibilidade
n_samples = 25               # Número de pontos de treino por dataset
n_datasets = 100             # Número de datasets para simulação Monte Carlo
n_test_points = 20           # Pontos de teste para avaliar o erro
noise_std = 0.5              # Desvio padrão do ruído adicionado aos dados
M = 1.0e3                    # Constante de escala da penalização
degree = 71                  # Grau máximo do polinômio
alpha_values = np.linspace(0.85, 1, 19)  # Valores de alpha (controlam complexidade)

# -------------------------------
# PREPARAÇÃO DOS PONTOS DE TESTE
# -------------------------------

x_test_points = np.random.rand(n_test_points)
X_test = poly_basis(x_test_points, degree)
f_real = f_true(x_test_points)

# Inicializa listas para armazenar os erros
bias2_list = []
var_list = []
noise_list = []

# -------------------------------
# LOOP SOBRE VALORES DE ALPHA
# -------------------------------

a_init = np.zeros(degree + 1)  # Inicialização comum para os coeficientes
for cnt, alpha in enumerate(alpha_values, start=1):
    preds = []  # Armazena predições em cada ponto para este alpha

    # Penalidade crescente nos coeficientes: M * alpha^k
    penalty_weights = M * np.array([alpha**k for k in range(degree + 1)])

    print(f"{cnt}/{len(alpha_values)} - α = {alpha:.4f}")

    for _ in range(n_datasets):
        # Gera dados sintéticos com ruído
        X_raw = np.linspace(0, 1, n_samples)
        y = f_true(X_raw) + np.random.normal(0, noise_std, n_samples)
        X_design = poly_basis(X_raw, degree)

        # Define a função de perda penalizada (soft regularization)
        def penalized_loss(a):
            residual = np.mean((y - X_design @ a) ** 2)
            penalty = np.sum((a / penalty_weights) ** 2)
            return residual + penalty

        # Minimiza a função de perda
        res = minimize(penalized_loss, x0=a_init, method='L-BFGS-B')

        # Atualiza os coeficientes ótimos
        a_opt = res.x
        a_init = a_opt  # Usar como ponto inicial da próxima iteração

        # Realiza predição nos pontos de teste
        y_pred = X_test @ a_opt
        preds.append(y_pred)

    # -------------------------------
    # DECOMPOSIÇÃO DO ERRO
    # -------------------------------

    preds = np.array(preds)
    bias2 = np.mean((np.mean(preds, axis=0) - f_real) ** 2)
    var = np.mean(np.var(preds, axis=0))
    noise = noise_std ** 2

    bias2_list.append(bias2)
    var_list.append(var)
    noise_list.append(noise)

# -------------------------------
# PLOTAGEM DOS RESULTADOS
# -------------------------------

plt.figure(figsize=(10, 6))
plt.plot(alpha_values, bias2_list, label='Viés²', linewidth=2)
plt.plot(alpha_values, var_list, label='Variância', linewidth=2)
plt.plot(alpha_values, np.array(bias2_list) + np.array(var_list) + noise_list, 
         label='Erro Total', linewidth=2, color='black')
plt.xlabel('Alpha (controle de complexidade)')
plt.ylabel('Erro Médio')
plt.title('Decomposição do Erro com Restrições Soft nos Coeficientes')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
"""
Experimento de Decomposição do Erro com Restrições Hard nos Coeficientes Polinomiais

Este script simula o comportamento do erro de generalização de um modelo de regressão polinomial
sob diferentes níveis de complexidade, controlados por um parâmetro de regularização alpha.

A complexidade do modelo é limitada por restrições explícitas (hard constraints) impostas sobre
os coeficientes do polinômio, de modo que |a_k| <= M * alpha^k.

O experimento estima a decomposição do erro médio quadrático esperado (EMQ) em viés², variância
e ruído, conforme alpha varia.

Autor: [Seu Nome]
Data: [Data]
Licença: MIT
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds

def f_true(x):
    """
    Função alvo verdadeira a ser aproximada.

    f(x) = 2x + cos(4πx)

    Parâmetros:
        x (array): entrada escalar ou vetorial

    Retorna:
        array: saída correspondente de f(x)
    """
    return 2 * x + np.cos(4 * np.pi * x)

def poly_basis(x, degree):
    """
    Gera a matriz de design para uma base polinomial de grau especificado.

    Parâmetros:
        x (array): vetor de entradas
        degree (int): grau máximo do polinômio

    Retorna:
        array: matriz de características (n x (degree+1))
    """
    return np.vstack([x**k for k in range(degree + 1)]).T

# ---------- Parâmetros do experimento ----------

np.random.seed(42)            # Reprodutibilidade
n_samples = 25                # Número de pontos de treino por dataset
n_datasets = 100              # Número de repetições para média/variância
n_test_points = 20            # Pontos para avaliação
noise_std = 0.5               # Desvio padrão do ruído aditivo
M = 1.0e3                     # Constante multiplicativa da restrição
degree = 71                  # Grau máximo do polinômio
alpha_values = np.linspace(0.2, 1.0, 51)  # Variação do parâmetro de regularização

# Pontos de teste (fixos) para avaliação
x_test_points = np.random.rand(n_test_points)
X_test = poly_basis(x_test_points, degree)
f_real = f_true(x_test_points)

# ---------- Armazenamento dos resultados ----------
bias2_list = []
var_list = []
noise_list = []

# ---------- Loop sobre diferentes valores de alpha ----------
a_init = np.zeros(degree + 1)  # Inicialização dos coeficientes

for i, alpha in enumerate(alpha_values):
    print(f"Executando {i+1}/{len(alpha_values)}: alpha = {alpha:.3f}")
    preds = []

    for _ in range(n_datasets):
        # Geração de dados sintéticos com ruído
        X_raw = np.linspace(0, 1, n_samples)
        y = f_true(X_raw) + np.random.normal(0, noise_std, n_samples)
        X_design = poly_basis(X_raw, degree)

        # Definição das restrições hard: |a_k| <= M * alpha^k
        lower_bounds = [-M * alpha**k for k in range(degree + 1)]
        upper_bounds = [ M * alpha**k for k in range(degree + 1)]
        bounds = Bounds(lower_bounds, upper_bounds)

        # Minimização do erro quadrático com restrições
        res = minimize(
            lambda a: np.mean((y - X_design @ a) ** 2),
            x0=a_init,
            bounds=bounds
        )

        a_opt = res.x
        a_init = a_opt  # Atualiza o ponto inicial para próxima execução
        y_pred = X_test @ a_opt
        preds.append(y_pred)

    # Conversão para array e cálculo da decomposição do erro
    preds = np.array(preds)
    bias2 = np.mean((np.mean(preds, axis=0) - f_real) ** 2)
    var = np.mean(np.var(preds, axis=0))
    noise = noise_std ** 2

    bias2_list.append(bias2)
    var_list.append(var)
    noise_list.append(noise)

# ---------- Visualização ----------

plt.figure(figsize=(10, 6))
plt.plot(alpha_values, bias2_list, label='Viés²')
plt.plot(alpha_values, var_list, label='Variância')
plt.plot(alpha_values, np.array(bias2_list) + np.array(var_list) + noise_list, label='Erro Total')
plt.xlabel('Alpha (Parâmetro de Regularização)')
plt.ylabel('Erro Médio')
plt.title('Decomposição do Erro com Restrições Hard nos Coeficientes')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
